# api

This is the primary interface to running squ wrappers 

In [ ]:
#| default_exp api

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import pandas, json, logging
from squ.core import *
from diskcache import memoize_stampede
from concurrent.futures import ThreadPoolExecutor
from importlib.resources import path
from subprocess import run

In [ ]:
#| export
logger = logging.basicConfig(level=logging.INFO)

## List Workspaces

The `list_workspaces` function retreives a list of workspaces from blob storage and returns it in various formats

In [ ]:
#| exports
@memoize_stampede(cache, expire=60 * 60 * 3) # cache for 3 hours
def list_workspaces(fmt: str = "df", # df, csv, json, list
                    agency: str = "ALL"): # Agency alias or ALL
    path = datalake_path()
    df = pandas.read_csv((path / "notebooks/lists/SentinelWorkspaces.csv").open())
    df = df.join(pandas.read_csv((path / "notebooks/lists/SecOps Groups.csv").open()).set_index("Alias"), on="SecOps Group", rsuffix="_secops")
    df = df.rename(columns={"SecOps Group": "alias", "Domains and IPs": "domains"})
    df = df.dropna(subset=["customerId"]).sort_values(by="alias")
    if agency != "ALL":
        df = df[df["alias"] == agency]
    if fmt == "df":
        return df
    elif fmt == "csv":
        return df.to_csv()
    elif fmt == "json":
        return df.fillna("").to_dict("records")
    elif fmt == "list":
        return list(df["customerId"].unique())
    else:
        raise ValueError("Invalid format")

In [ ]:
list_workspaces().head()

,DIRECTORY,ITSA Email,Information Classification,JiraOrgId,LOCATION,M365 Security Portal,RESOURCE GROUP,alias,SecOps Status,Subscription Name,...,Email Identities,Full Time Employee,ITSA Email_secops,JiraOrgId_secops,Primary Agency,Primary Agency Type,Risk Profile,SOC Intent,Status,Target SLA
39,Aqwest,it_alerts@aqwest.com.au,OFFICIAL,36.0,Australia Central,NaN,rg_aqwestcorp_azureau_central,Aqwest,Connected:T0; Risk:High - Critical Infrastructure,Aqwest Bunbury Water Corp Azure,...,NaN,42.0,it_alerts@aqwest.com.au,36.0,Aqwest,Schedule 1 Entity,High - Critical Infrastructure,Connected,Connected:T0; Risk:High - Critical Infrastructure,NaN
47,Construction Training Fund,emoore@bcitf.org,OFFICIAL,49.0,Australia East,NaN,ctf-prd-sentinel-rg,CTF,Connected:T0; Risk:Low,CTF - PROD,...,jbertram@ctf.wa.gov.au,30.0,emoore@bcitf.org,49.0,Construction Training Fund,Non-SES Entity,Low,Connected,Connected:T0; Risk:Low,SLA1
31,ChemCentre,itsa@chemcentre.wa.gov.au,OFFICIAL,32.0,Australia East,NaN,ccwa_au_ea_rg1,ChemCentre,Connected:T0; Risk:Medium,PAYG-PG_CC,...,NaN,140.0,itsa@chemcentre.wa.gov.au,32.0,Chemistry Centre (WA),SES Organisation (Schedule 2),Medium,Connected,Connected:T0; Risk:Medium,NaN
0,"Department of Biodiversity, Conservation and A...",ITSA@dbca.wa.gov.au,OFFICIAL,3.0,Australia Southeast,https://security.microsoft.com/v2/advanced-hun...,oim-appservices,DBCA,Connected:T1; Risk:Medium,OIM Azure Subscription,...,"chris.hocking@dbca.wa.gov.au, brendan.cale@dbc...",2046.0,ITSA@dbca.wa.gov.au,3.0,"Department of Biodiversity, Conservation and A...",Department (Section 35),Medium,Connected,Connected:T1; Risk:Medium,SLA1
8,Department of Fire and Emergency Services,ictsa@dfes.wa.gov.au,OFFICIAL,22.0,Australia East,NaN,azaue-sentinel-prod-01-rg,DFES,Connected:T0; Risk:High - Emergency Services,DFES-Production-EA,...,sami.anderson@dfes.wa.gov.au\nsimon.rice@dfes....,1673.0,ictsa@dfes.wa.gov.au,22.0,Department of Fire and Emergency Services,Department (Section 35),High - Emergency Services,Connected,Connected:T0; Risk:High - Emergency Services,NaN


# Log Analytics Query
The below function makes it easy to query all workspaces with sentinel installed using log analytics.

In [ ]:
#| exports
@memoize_stampede(cache, expire=60 * 60 * 3) # cache for 3 hours
def list_subscriptions():
    return pandas.DataFrame(azcli(["account", "list"]))["id"].unique()

@memoize_stampede(cache, expire=60 * 60 * 3) # cache for 3 hours
def list_securityinsights():
    return pandas.DataFrame(azcli([
        "graph", "query", "--first", "1000", "-q", 
        """
        resources
        | where type =~ 'microsoft.operationsmanagement/solutions'
        | where name startswith 'SecurityInsights'
        | project wlid = tolower(tostring(properties.workspaceResourceId))
        | join kind=leftouter (
            resources | where type =~ 'microsoft.operationalinsights/workspaces' | extend wlid = tolower(id))
            on wlid
        | extend customerId = properties.customerId
        """
    ])["data"])

def loganalytics_query(query: str):
    dfs = []
    customerids = list_securityinsights()["customerId"]
    with ThreadPoolExecutor(max_workers=32) as executor:
        futures = [executor.submit(azcli, [
            "monitor", "log-analytics", "query",
            "-w", workspace,
            "--analytics-query", query
        ]) for workspace in customerids]
        for future, customerid in zip(futures, customerids):
            try:
                df = pandas.DataFrame(future.result())
            except Exception as e:
                logger.warning(e)
                continue
            else:
                if "TenantId" not in df.columns:
                    df["TenantId"] = customerid
                dfs.append(df)
    return pandas.concat(dfs)

def query_all(query: str, fmt="df"):
    df = loganalytics_query(query)
    if fmt == "df":
        return df
    elif fmt == "csv":
        return df.to_csv()
    elif fmt == "json":
        return df.fillna("").to_dict("records")
    else:
        raise ValueError("Invalid format")

In [ ]:
list_securityinsights()

,customerId,extendedLocation,id,identity,kind,location,managedBy,name,plan,properties,resourceGroup,sku,subscriptionId,tags,tenantId,type,wlid,wlid1,zones
0,1f532cb7-2ac7-47c6-b892-e738ebcca160,None,/subscriptions/01fb6fc8-91da-4fdd-b103-a54c330...,None,,australiaeast,,Sentinel-Workspace,None,{'createdDate': '2022-07-21T06:13:53.8397694Z'...,sentinel,None,01fb6fc8-91da-4fdd-b103-a54c330efd97,{},308d2158-f6df-4747-8791-e970657274d5,microsoft.operationalinsights/workspaces,/subscriptions/01fb6fc8-91da-4fdd-b103-a54c330...,/subscriptions/01fb6fc8-91da-4fdd-b103-a54c330...,None
1,a41c5c34-0105-4fb9-898a-8aa33f679945,None,/subscriptions/0b0e2ab4-b76d-4abe-9f04-b2dec10...,None,,australiaeast,,WAEC-HUB-AUE-LAW-001,None,{'createdDate': '2022-05-26T04:17:22.5250488Z'...,waec-hub-aue-rg-logs-001,None,0b0e2ab4-b76d-4abe-9f04-b2dec10bf38f,"{'Department': 'IT', 'Environment': 'HUB'}",adb28173-f332-4dc4-a6d5-8b995daa2c9b,microsoft.operationalinsights/workspaces,/subscriptions/0b0e2ab4-b76d-4abe-9f04-b2dec10...,/subscriptions/0b0e2ab4-b76d-4abe-9f04-b2dec10...,None
2,07d6e677-8597-4676-9598-298a53c172df,None,/subscriptions/0f62b1fe-3557-467a-a6f8-d9c98e3...,None,,australiaeast,,ae-sh-la-001,None,{'createdDate': '2020-05-11T06:38:57.0000000Z'...,ae-sh-rg-la,None,0f62b1fe-3557-467a-a6f8-d9c98e3589f8,"{'AppName': 'Microsoft Azure', 'AppSysId': 'f3...",9177d956-8388-4f68-ae34-ef983dc9e160,microsoft.operationalinsights/workspaces,/subscriptions/0f62b1fe-3557-467a-a6f8-d9c98e3...,/subscriptions/0f62b1fe-3557-467a-a6f8-d9c98e3...,None
3,c84edfef-4bfb-493d-88bc-0b917c542fdc,None,/subscriptions/0f62b1fe-3557-467a-a6f8-d9c98e3...,None,,australiaeast,,SecureScoreData-7mejwboxra5wo,None,{'createdDate': '2020-09-07T23:47:22.0000000Z'...,ae-sh-rg-la,None,0f62b1fe-3557-467a-a6f8-d9c98e3589f8,"{'AppName': 'Microsoft Azure', 'AppSysId': 'f3...",9177d956-8388-4f68-ae34-ef983dc9e160,microsoft.operationalinsights/workspaces,/subscriptions/0f62b1fe-3557-467a-a6f8-d9c98e3...,/subscriptions/0f62b1fe-3557-467a-a6f8-d9c98e3...,None
4,2c3382ca-844d-4e94-be3f-dbadbd0b53ae,None,/subscriptions/0f62b1fe-3557-467a-a6f8-d9c98e3...,None,,australiaeast,,SecureScoreData-bhbrj5vyuzmbk,None,{'createdDate': '2020-07-09T03:41:51.0000000Z'...,ae-sh-rg-la,None,0f62b1fe-3557-467a-a6f8-d9c98e3589f8,"{'AppName': 'Microsoft Azure', 'AppSysId': 'f3...",9177d956-8388-4f68-ae34-ef983dc9e160,microsoft.operationalinsights/workspaces,/subscriptions/0f62b1fe-3557-467a-a6f8-d9c98e3...,/subscriptions/0f62b1fe-3557-467a-a6f8-d9c98e3...,None
5,26fe9edf-c4de-49b6-ac68-a2a00aee43b2,None,/subscriptions/0f62b1fe-3557-467a-a6f8-d9c98e3...,None,,australiasoutheast,,ase-sh-la-002,None,{'createdDate': '2019-10-29T06:45:14.0000000Z'...,ase-sh-rg-la,None,0f62b1fe-3557-467a-a6f8-d9c98e3589f8,"{'AppName': 'Microsoft Azure', 'AppSysId': 'f3...",9177d956-8388-4f68-ae34-ef983dc9e160,microsoft.operationalinsights/workspaces,/subscriptions/0f62b1fe-3557-467a-a6f8-d9c98e3...,/subscriptions/0f62b1fe-3557-467a-a6f8-d9c98e3...,None
6,7cf7048e-d494-4a98-8bf3-a60fb27baae4,None,/subscriptions/1018ace2-39f1-4bc6-81a5-99fad1a...,None,,australiaeast,,GC-SYD-COR-OMS-001,None,{'createdDate': '2019-08-21T01:53:32.0000000Z'...,gc-cor-arg-administration,None,1018ace2-39f1-4bc6-81a5-99fad1a2382d,"{'Application': 'AvDC-Foundations', 'Environme...",cc85cee5-e5c8-4c93-b93a-d12419e89c07,microsoft.operationalinsights/workspaces,/subscriptions/1018ace2-39f1-4bc6-81a5-99fad1a...,/subscriptions/1018ace2-39f1-4bc6-81a5-99fad1a...,None
7,af1333b1-0288-48e9-aa83-a65fa43e50df,None,/subscriptions/1018ace2-39f1-4bc6-81a5-99fad1a...,None,,australiaeast,,GC-SYD-COR-OMS-002,None,{'createdDate': '2022-08-08T00:45:08.7914169Z'...,gc-cor-arg-siem,None,1018ace2-39f1-4bc6-81a5-99fad1a2382d,"{'Application': 'MS Sentinel', 'Environment': ...",cc85cee5-e5c8-4c93-b93a-d12419e89c07,microsoft.operationalinsights/workspaces,/subscriptions/1018ace2-39f1-4bc6-81a5-99fad1a...,/subscriptions/1018ace2-39f1-4bc6-81a5-99fad1a...,None
8,faa02a1f-84c6-4442-92b6-d71e

In [ ]:
df = query_all("""
SecurityIncident
| where TimeGenerated > ago(45d)
| where Classification == "TruePositive"
| mv-expand AlertIds
| project tostring(AlertIds)
| join SecurityAlert on $left.AlertIds == $right.SystemAlertId
| mv-expand todynamic(Entities)
| project Entities.Address
| where isnotempty(Entities_Address)
| distinct tostring(Entities_Address)
""")

In [ ]:
df.shape

(1327, 3)

In [ ]:
def hunt(iocs=list[str]):
    hunt_text = '" or "'.join(iocs)
    hunt_text = f'"{hunt_text}"'
    if len(iocs) > 1:
        hunt_text = f"({hunt_text})"
    query = f'search {hunt_text} and TimeGenerated > ago(14d) | summarize hits = count() by $table'
    print(query)
    return query_all(query)

# hunt(["91.191.209.190", "196.216.136.139", "66.203.112.86"])

In [ ]:
hunt(df.Entities_Address.unique()[:100])

search ("91.191.209.190" or "110.248.108.255" or "111.36.163.160" or "14.208.172.126" or "211.51.62.226" or "218.11.102.79" or "42.236.216.58" or "218.10.17.144" or "61.178.32.114" or "221.193.151.152" or "177.125.72.96" or "120.230.24.212" or "223.91.89.56" or "154.13.44.72" or "194.169.175.93" or "49.89.179.245" or "120.82.83.73" or "183.213.121.59" or "219.144.219.147" or "120.229.231.204" or "111.226.162.63" or "111.230.241.104" or "183.193.168.2" or "115.151.39.24" or "170.64.134.89" or "107.170.249.23" or "95.214.27.204" or "192.241.206.21" or "198.199.118.8" or "170.64.134.120" or "210.160.217.69" or "170.64.166.144" or "198.74.56.46" or "192.241.198.38" or "162.243.147.15" or "159.203.240.10" or "207.244.235.11" or "107.170.252.26" or "192.241.235.23" or "167.99.13.19" or "206.189.120.50" or "107.170.231.9" or "138.68.143.68" or "64.227.146.243" or "49.229.22.10" or "146.190.154.101" or "192.241.206.20" or "104.248.127.48" or "64.227.41.39" or "167.172.89.248" or "64.226.68.166

AttributeError: 'NoneType' object has no attribute 'warning'

In [ ]:
#| exports

def atlaskit_transformer(inputtext, inputfmt="md", outputfmt="wiki", runtime="node", transformer=path("squ", "atlaskit-transformer.bundle.js").absolute()):
    return run([runtime, transformer, inputfmt, outputfmt], input=inputtext, text=True, capture_output=True, check=True).stdout

In [ ]:
print(atlaskit_transformer("""# Heading 1

- a bullet
- [a link](https://github.com)
"""))

h1. Heading 1

* a bullet
* [a link|https://github.com]



In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()